# Example 4: Save Model Inputs and Outputs

This notebook demonstrates how to:
1. Load a language model
2. Attach `ModelInputDetector` and `ModelOutputDetector` to capture tokenized inputs and model outputs
3. Run inference and save the captured data
4. Access and inspect the saved inputs and outputs

The detectors are useful for:
- Saving complete input-output pairs for analysis
- Creating datasets for fine-tuning or training
- Debugging model behavior
- Analyzing model responses to specific inputs

**Note:** `ModelInputDetector` uses PRE_FORWARD hook to capture inputs, while `ModelOutputDetector` uses FORWARD hook to capture outputs.


In [38]:
# Setup and imports
%load_ext autoreload
%autoreload 2

import torch
from pathlib import Path
from datetime import datetime

from amber.hooks import ModelInputDetector, ModelOutputDetector
from amber.language_model.language_model import LanguageModel
from amber.store.local_store import LocalStore

print("✅ Imports completed")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Imports completed


In [39]:
# Configuration
MODEL_ID = "sshleifer/tiny-gpt2"  # Small model for quick experimentation
STORE_DIR = Path("store")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Test texts to process
TEST_TEXTS = [
    "The quick brown fox jumps over the lazy dog.",
    "Machine learning is a subset of artificial intelligence.",
    "Python is a popular programming language.",
]

print("🚀 Starting Input/Output Saving Example")
print(f"📱 Using device: {DEVICE}")
print(f"🔧 Model: {MODEL_ID}")
print(f"📝 Number of test texts: {len(TEST_TEXTS)}")
print()

STORE_DIR.mkdir(parents=True, exist_ok=True)
print("✅ Output directories created")


🚀 Starting Input/Output Saving Example
📱 Using device: cpu
🔧 Model: sshleifer/tiny-gpt2
📝 Number of test texts: 3

✅ Output directories created


In [40]:
# Step 1: Load language model
print("📥 Loading language model...")

store = LocalStore(STORE_DIR)
lm = LanguageModel.from_huggingface(MODEL_ID, store=store)
lm.model.to(DEVICE)

print(f"✅ Model loaded: {lm.model_id}")
print(f"📱 Device: {DEVICE}")
print(f"📁 Store location: {lm.context.store.base_path}")


📥 Loading language model...
✅ Model loaded: sshleifer_tiny-gpt2
📱 Device: cpu
📁 Store location: store


In [41]:
# Step 2: Create and attach separate input and output detectors
print("🔧 Creating ModelInputDetector and ModelOutputDetector...")

# Use different layer signatures for input and output detectors
# This ensures they save to separate subfolders in the store
input_layer_signature = "model_inputs"
output_layer_signature = "model_outputs"

# Get the root model for hooking
root_model = lm.model

# Add both layer signatures to the layers registry
if input_layer_signature not in lm.layers.name_to_layer:
    lm.layers.name_to_layer[input_layer_signature] = root_model
    print(f"📝 Added '{input_layer_signature}' to layers registry")

if output_layer_signature not in lm.layers.name_to_layer:
    lm.layers.name_to_layer[output_layer_signature] = root_model
    print(f"📝 Added '{output_layer_signature}' to layers registry")

# Create input detector (uses PRE_FORWARD hook)
input_detector = ModelInputDetector(
    layer_signature=input_layer_signature,
    hook_id="model_input_detector",
    save_input_ids=True,
    save_attention_mask=False,  # Set to True if you need attention masks
)

# Create output detector (uses FORWARD hook)
output_detector = ModelOutputDetector(
    layer_signature=output_layer_signature,
    hook_id="model_output_detector",
    save_output_logits=True,
    save_output_hidden_state=False,  # Set to True if you need hidden states
)

# Register hooks using the layers system
input_hook_id = lm.layers.register_hook(input_layer_signature, input_detector)
output_hook_id = lm.layers.register_hook(output_layer_signature, output_detector)

print(f"✅ Detectors attached to model via layers system")
print(f"🆔 Input detector ID: {input_detector.id}")
print(f"🆔 Output detector ID: {output_detector.id}")
print(f"💾 Will save: input_ids, output_logits")


🔧 Creating ModelInputDetector and ModelOutputDetector...
📝 Added 'model_inputs' to layers registry
📝 Added 'model_outputs' to layers registry
✅ Detectors attached to model via layers system
🆔 Input detector ID: model_input_detector
🆔 Output detector ID: model_output_detector
💾 Will save: input_ids, output_logits


In [42]:
# Step 3: Run inference and capture inputs/outputs
print("🚀 Running inference...")
print(f"📝 Processing {len(TEST_TEXTS)} texts")

# Clear any previous captures
input_detector.clear_captured()
output_detector.clear_captured()

# Run forward pass
output, encodings = lm.forwards(
    TEST_TEXTS,
    tok_kwargs={"max_length": 128, "padding": True, "truncation": True},
    autocast=False,
)

# Note: For HuggingFace models called with **kwargs, the pre_forward hook
# doesn't receive the inputs. We need to manually set them from encodings.
input_detector.set_inputs_from_encodings(encodings)

print("✅ Inference completed")
print(f"📊 Output type: {type(output)}")
print(f"📊 Encodings keys: {list(encodings.keys()) if isinstance(encodings, dict) else 'N/A'}")
print()
print("💡 Data captured in detectors - ready to save to store")


🚀 Running inference...
📝 Processing 3 texts
✅ Inference completed
📊 Output type: <class 'transformers.modeling_outputs.CausalLMOutputWithCrossAttentions'>
📊 Encodings keys: ['input_ids', 'attention_mask']

💡 Data captured in detectors - ready to save to store


In [43]:
# Step 4: Inspect captured inputs and outputs
print("🔍 Inspecting captured data...")

# Get captured input_ids from input detector
input_ids = input_detector.get_captured_input_ids()
if input_ids is not None:
    print(f"✅ Captured input_ids (from ModelInputDetector):")
    print(f"   Shape: {input_ids.shape}")
    print(f"   Dtype: {input_ids.dtype}")
    print(f"   Sample (first 10 tokens of first text): {input_ids[0, :10].tolist()}")
    print()
else:
    print("❌ No input_ids captured")

# Get captured output logits from output detector
output_logits = output_detector.get_captured_output_logits()
if output_logits is not None:
    print(f"✅ Captured output_logits (from ModelOutputDetector):")
    print(f"   Shape: {output_logits.shape}")
    print(f"   Dtype: {output_logits.dtype}")
    print(f"   Vocabulary size: {output_logits.shape[-1]}")
    print()
else:
    print("❌ No output_logits captured")

# Show metadata from both detectors
print("📋 Input detector metadata:")
for key, value in input_detector.metadata.items():
    print(f"   {key}: {value}")

print("\n📋 Output detector metadata:")
for key, value in output_detector.metadata.items():
    print(f"   {key}: {value}")


🔍 Inspecting captured data...
✅ Captured input_ids (from ModelInputDetector):
   Shape: torch.Size([3, 10])
   Dtype: torch.int64
   Sample (first 10 tokens of first text): [464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290, 13]

✅ Captured output_logits (from ModelOutputDetector):
   Shape: torch.Size([3, 10, 50257])
   Dtype: torch.float32
   Vocabulary size: 50257

📋 Input detector metadata:
   input_ids_shape: (3, 10)

📋 Output detector metadata:
   output_logits_shape: (3, 10, 50257)


In [44]:
# Step 5: Decode and verify the data
print("🔤 Decoding captured data...")

if input_ids is not None:
    print("\n📝 Decoded input texts:")
    for i, text in enumerate(TEST_TEXTS):
        decoded = lm.tokenizer.decode(input_ids[i], skip_special_tokens=False)
        print(f"   Text {i+1}: {text}")
        print(f"   Decoded: {decoded}")
        print()

if output_logits is not None:
    print("🎯 Predicted tokens (argmax of logits):")
    predicted_token_ids = output_logits.argmax(dim=-1)
    for i in range(len(TEST_TEXTS)):
        # Get the last predicted token for each sequence
        last_token_id = predicted_token_ids[i, -1].item()
        predicted_token = lm.tokenizer.decode([last_token_id], skip_special_tokens=True)
        print(f"   Text {i+1} last predicted token: {predicted_token}")


🔤 Decoding captured data...

📝 Decoded input texts:
   Text 1: The quick brown fox jumps over the lazy dog.
   Decoded: The quick brown fox jumps over the lazy dog.

   Text 2: Machine learning is a subset of artificial intelligence.
   Decoded: Machine learning is a subset of artificial intelligence.<|endoftext|>

   Text 3: Python is a popular programming language.
   Decoded: Python is a popular programming language.<|endoftext|><|endoftext|><|endoftext|>

🎯 Predicted tokens (argmax of logits):
   Text 1 last predicted token:  factors
   Text 2 last predicted token:  factors
   Text 3 last predicted token:  factors


In [45]:
# Step 6: Access tensor metadata directly
print("💾 Accessing all captured tensors...")

print("Available tensors in input_detector.tensor_metadata:")
for key, tensor in input_detector.tensor_metadata.items():
    print(f"   {key}: shape={tensor.shape}, dtype={tensor.dtype}")

print("\nAvailable tensors in output_detector.tensor_metadata:")
for key, tensor in output_detector.tensor_metadata.items():
    print(f"   {key}: shape={tensor.shape}, dtype={tensor.dtype}")

print("\n✅ All data is stored in detector.tensor_metadata")
print("   You can access it directly or save it to disk as needed")


💾 Accessing all captured tensors...
Available tensors in input_detector.tensor_metadata:
   input_ids: shape=torch.Size([3, 10]), dtype=torch.int64

Available tensors in output_detector.tensor_metadata:
   output_logits: shape=torch.Size([3, 10, 50257]), dtype=torch.float32

✅ All data is stored in detector.tensor_metadata
   You can access it directly or save it to disk as needed


In [46]:
# Step 7: Save detector outputs to store
print("💾 Saving detector outputs to store...")

# Create a run name for this batch
run_name = f"model_io_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
batch_idx = 0

# Save detector metadata (this saves both input and output detector data)
saved_path = lm.save_detector_metadata(run_name, batch_idx)

print(f"✅ Detector outputs saved to store")
print(f"📁 Run name: {run_name}")
print(f"📁 Batch index: {batch_idx}")
print(f"📁 Saved path: {saved_path}")
print()
print("💡 The saved data includes:")
print("   - input_ids tensor (from ModelInputDetector)")
print("   - output_logits tensor (from ModelOutputDetector)")
print("   - All metadata (shapes, etc.)")


💾 Saving detector outputs to store...
✅ Detector outputs saved to store
📁 Run name: model_io_20251126_221443
📁 Batch index: 0
📁 Saved path: runs/model_io_20251126_221443/batch_0

💡 The saved data includes:
   - input_ids tensor (from ModelInputDetector)
   - output_logits tensor (from ModelOutputDetector)
   - All metadata (shapes, etc.)


In [47]:
# Step 8: Verify saved data by loading it back
print("🔍 Verifying saved data...")

# Load the saved detector metadata
retrieved_metadata, retrieved_tensors = lm.store.get_detector_metadata(run_name, batch_idx)

print(f"✅ Loaded metadata for {len(retrieved_metadata)} layer(s)")
print(f"✅ Loaded tensors for {len(retrieved_tensors)} layer(s)")

# Check what was saved for input and output layers
print(f"\n📦 Saved tensors:")
for layer_sig in [input_layer_signature, output_layer_signature]:
    if layer_sig in retrieved_tensors:
        print(f"   Layer '{layer_sig}':")
        for tensor_key, tensor in retrieved_tensors[layer_sig].items():
            print(f"      {tensor_key}: shape={tensor.shape}, dtype={tensor.dtype}")

print(f"\n📋 Saved metadata:")
for layer_sig in [input_layer_signature, output_layer_signature]:
    if layer_sig in retrieved_metadata:
        print(f"   Layer '{layer_sig}':")
        for key, value in retrieved_metadata[layer_sig].items():
            print(f"      {key}: {value}")


🔍 Verifying saved data...
✅ Loaded metadata for 2 layer(s)
✅ Loaded tensors for 2 layer(s)

📦 Saved tensors:
   Layer 'model_inputs':
      input_ids: shape=torch.Size([3, 10]), dtype=torch.int64
   Layer 'model_outputs':
      output_logits: shape=torch.Size([3, 10, 50257]), dtype=torch.float32

📋 Saved metadata:
   Layer 'model_inputs':
      input_ids_shape: [3, 10]
   Layer 'model_outputs':
      output_logits_shape: [3, 10, 50257]


In [48]:
# Step 9: Cleanup - remove hooks
print("🧹 Cleaning up...")

# Unregister hooks using the layers system
lm.layers.unregister_hook(input_hook_id)
lm.layers.unregister_hook(output_hook_id)

# Clear captured data
input_detector.clear_captured()
output_detector.clear_captured()

print("✅ Hooks removed and data cleared")
print("\n💡 Tip: You can process multiple batches by:")
print("   1. Calling detector.clear_captured() on both detectors")
print("   2. Re-running inference")
print("   3. Calling lm.save_detector_metadata(run_name, batch_idx) for each batch")
print("\n💡 Note: You can use input and output detectors independently:")
print("   - Use only ModelInputDetector if you only need inputs")
print("   - Use only ModelOutputDetector if you only need outputs")


🧹 Cleaning up...
✅ Hooks removed and data cleared

💡 Tip: You can process multiple batches by:
   1. Calling detector.clear_captured() on both detectors
   2. Re-running inference
   3. Calling lm.save_detector_metadata(run_name, batch_idx) for each batch

💡 Note: You can use input and output detectors independently:
   - Use only ModelInputDetector if you only need inputs
   - Use only ModelOutputDetector if you only need outputs
